## Implicit feedbackを想定し、バイアスを考慮した推薦　　
### データが増えるとその分どのくらいバイアスに気を使わないといけないのか調査
人気順推薦から生まれるランキングバイアスパラメータを特定し、IPS推定量とNaive推定量を比較  
IPS推定量の分散が大きければ、ロバスト推定量を使用したいところ

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import random

class train_test_split():
    
    def __init__(self) -> None:
        self.feature = []
        self.item_id = []
        self.populality = []

    def make_dataframe(self) -> pd.DataFrame:
        item_dict = {"item_id": self.item_id, "feature": self.feature, "populality": self.populality}
        df = pd.DataFrame(data=item_dict)
        
        return df
        
    def create_dataset(self) -> (pd.DataFrame, pd.DataFrame):
    
        for i in range(100):
            self.feature.append(round(random.uniform(1,5),2))
            self.populality.append(round(random.uniform(1,5),2))
            self.item_id.append(str(i+1))
        
            if i == 69:
                df_train = self.make_dataframe()
                self.feature, self.item_id, self.populality = [], [], []
        
        df_test = self.make_dataframe()
        
        return df_train, df_test
                

# 特徴量の説明
feature:  ユーザーのクリックを決める特徴量(5に近いとクリックしやすい)  
populality:  一ヶ月ごとの人気度を表す特徴量(5に近いと推薦されやすい。人気順の推薦を想定)  
また、train, testは7:3

In [3]:
df_train, df_test = train_test_split().create_dataset()

In [4]:
df_train.head()

,item_id,feature,populality
0,1,3.47,3.84
1,2,4.89,2.73
2,3,1.17,1.88
3,4,2.92,2.02
4,5,3.53,3.04


In [5]:
# ポジションバイアスはE[O(u,i)]としpow_trueパラメータを0.5とする
# C(u,i,k) = O(u,i)*R(u,i)
# E[C(u,i,k)] = THETA(u,i)*r(u,i)

position_bias = lambda k: pow(0.9/k, 0.2) 

In [6]:
# r(u,i)をfeatureとし、0-1スケールに変換。featureはあくまで特徴量なので、
#誤クリックを加えることで過学習を防ぐ

In [7]:
def user_feedback(df, policy) -> pd.DataFrame:
    # 人気順orランダム推薦でランキングしてみる（毎月のバッチ処理と仮定）
    # 数値の高い順にすると、ログが同じになるので、少しランダムに
    sorted_df = df.copy()
    if policy == "populality":
        sorted_df = sorted_df[sorted_df["populality"] >= 3.8]
        sorted_df.reset_index(drop=True, inplace=True)
    position = list(range(1,8))
    item_dict = {
        "position": [], "item_id": [], 
        "clicked": [], "iter": []
    }
    for i in range(50):
        recommend_items = random.sample(list(sorted_df.index.values), k=7)
        df = sorted_df.loc[recommend_items,:]
        click_true = np.zeros(7);
        rel_max = 5
        rel = 0.1 + 0.9 * pow(2, df["feature"].values-1)/pow(2, rel_max-1)
    
        for k in range(len(click_true)):
            theta = position_bias(k+1)
            E_click = rel[k]#*theta
            if (E_click >= 0.7):
                click_true[k] = 1
            #else:
                # 誤クリック
                #if (random.random() < 0.1):
                 #   click_true[k] = 1
        
        item_dict["position"].extend(position)
        item_dict["item_id"].extend(df["item_id"].values)
        item_dict["clicked"].extend(click_true)
        item_dict["iter"].extend(len(position)*[i+1])
    
    log_df = pd.DataFrame(data=item_dict)
    log_df = log_df.astype({"clicked": np.int64})
    
    count_dict = log_df[log_df["clicked"]==1].groupby("item_id").agg({"clicked": "count"})["clicked"].to_dict()
    history_dict = {"item_id": list(count_dict.keys()), "click count": list(count_dict.values())}
    user_df = pd.DataFrame(data=history_dict)
    
    return log_df, user_df

In [8]:
train_log_df, train_user_df = user_feedback(df_train, policy="random")
test_log_df, test_user_df = user_feedback(df_test, policy="random")

In [9]:
#df[df["item_id"].isin(user_df["item_id"].values)]

In [10]:
for_cosine_df = pd.merge(train_log_df, df_train, on="item_id")
for_cosine_df = pd.concat([for_cosine_df, df_test], sort=True)[["item_id", "feature", "clicked"]]
# テストデータのクリックを-1とする
for_cosine_df = for_cosine_df.fillna({"clicked": -1.0})
for_cosine_df = for_cosine_df.astype({"clicked": np.int64})


### 類似度でとりあえずnaiveに推薦してみる

In [11]:
def cosine_recommend(df) -> list:
    profile = round(df[df["clicked"]==1]["feature"].mean(),3)
    print(f'ユーザプロファイル: {profile}')
    print('-------------------------')
    rec_list = []
    for i, row in df[df["clicked"]==-1].iterrows():
        similarity = round(np.abs(row["feature"]-profile),2)
        item_vec = (row["item_id"], similarity)
        rec_list.append(item_vec)
    
    rec_list.sort(key=lambda x:x[1])
    rec_list = rec_list[:7]
    
    print(f'推薦リスト. (item_is, ベクトルの差): {rec_list}')
    
    rec_list = [list(tup) for tup in zip(*rec_list)][0]
    
    return rec_list

In [12]:
rec_list = cosine_recommend(for_cosine_df)

ユーザプロファイル: 4.716
-------------------------
推薦リスト. (item_is, ベクトルの差): [('83', 0.01), ('91', 0.04), ('92', 0.04), ('98', 0.07), ('87', 0.08), ('71', 0.16), ('93', 0.21)]


特徴量"feature"が5に近いほどこのユーザはクリックしやすい傾向にあるにも関わらず、クリックしたときの平均値が3.9  
である。すなわち、ポジションバイアスと誤クリックの影響を考慮できてない

### Map@7で評価する

In [13]:
# 各レコードの平均 precision
# 1/min(m,K)*p

def apk(y_i_true, y_i_pred):
    assert (len(y_i_pred) <= 7)
    assert (len(np.unique(y_i_pred)) == len(y_i_pred))
    
    sum_precision = 0.0
    num_hits = 0
    
    for i, p in enumerate(y_i_pred):
        if p in y_i_true:
            num_hits += 1
            precision = num_hits / (i+1)
            sum_precision += precision
        
    if sum_precision == 0.0:
        return 0.0
    
    else :
        return sum_precision / min(len(y_i_true), 7)

#MAP@K計算用の関数
def mapk(y_true, y_pred):
    ap = [apk(y_i_true, y_i_pred) for y_i_true, y_i_pred in zip(y_true, y_pred)]
    
    return np.mean(ap)
                   


In [14]:
def devide_df(df):
    devided_list = []
    for i in range(50):
        clicked_items = df[(df["iter"]==i) & (df["clicked"]==1)]["item_id"].to_list()
        devided_list.append(clicked_items)
    
    return devided_list

y_true = devide_df(test_log_df)

In [15]:
y_pred = [rec_list]*50

In [16]:
naive_map = mapk(y_true, y_pred)
print(f'MAP@7 : {round(naive_map,2)}')

MAP@7 : 0.45


In [17]:
## ポジションバイアス込みにすると、mapは0.05とかになる